# Ensemble Regression

In [ ]:
# modules
import random as rd  # generating random numbers for distributions
import pandas as pd  # data mangling and transforming
import numpy as np  # handling vectors and matrices
import matplotlib.pyplot as plt  # plotting module
import seaborn as sns  # pairplot & heatmap for correlations
import graphviz  # dendograms
import sympy  # linear dependencies
import torch  # neural net

from math import sqrt  # mathematical operators
from sklearn import tree, linear_model  # lm + regression & classification trees
from sklearn.preprocessing import StandardScaler, MinMaxScaler, Normalizer, PolynomialFeatures # preprocessing tools
from sklearn.feature_selection import VarianceThreshold, RFE, SelectFromModel
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, ExtraTreesClassifier, AdaBoostRegressor
from sklearn.metrics import mean_squared_error, r2_score

import torch
import torch.nn.functional as F
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader
from torch.autograd import Variable

In [ ]:
# plot inside notebook
get_ipython().run_line_magic('matplotlib', 'inline')

## Test data

In [3]:
n = 60000

In [4]:
df = pd.DataFrame(data={'MTTF': np.random.choice(range(0, 3500), n),
                        'longitude': np.random.choice(range(10, 50), n),
                        'latitude': np.random.choice(range(10, 50), n),
                        'A': np.append(np.random.choice(range(1, 5), n-1), np.nan),
                        'B': np.random.choice(range(1, 500000), n),
                        'C': np.random.choice(range(0, 50000), n),
                        'D': np.random.choice(['Sam', 'Som', 'Sim'], n),
                        'E': np.random.choice([''.join(rd.choices(string.ascii_uppercase, k=5)) 
                                               for i in range(10)], n),
                       })

## Helper Functions

In [ ]:
# create new dataframe after removing variables with variance below threshold
def varianceSelection(X, THRESHOLD = .1):
    sel = VarianceThreshold(threshold=THRESHOLD)
    sel.fit_transform(X)
    return X[[c for (s, c) in zip(sel.get_support(), X.columns.values) if s]]


# calculate own score robustly
def calc_zscore(col, ddof=0):
    return (col - col.mean())/col.std(ddof=ddof)

In [ ]:
## Dataset(s)

### inverter

In [ ]:
# read data
df = pd.read_csv('inverters.csv', sep=';')

In [ ]:
# change index to be udid
df.index = df.UDID

In [ ]:
# additional columns
for v in ['MEAN', 'MEDIAN', 'MIN', 'MAX']:
    df.loc[:,'ACDC_RATIO_'+v] = df.loc[:,'AC_POWER_'+v] / df.loc[:,'DC_POWER_'+v]
    df.loc[:,'AC_POWER_NORM_'+v] = df.loc[:,'AC_POWER_'+v] / df.loc[:,'POWER_NOMINAL']
    df.loc[:,'DC_POWER_NORM_'+v] = df.loc[:,'DC_POWER_'+v] / df.loc[:,'POWER_NOMINAL']
    df.loc[:,'TEMP_DIFF_'+v] = df.loc[:,'TEMPERATURE_'+v] / df.loc[:,'FELTTEMPERATURE_'+v]

In [ ]:
# keep track of variable with most missings by creating a dummy
df['DC_MISSING_DUMMY'] = df.DC_VOLTAGE_MEAN.isnull()*1

In [ ]:
# categorization of columns
names_drop = ['UDID', 'DEVICE_TYPE']
names_y1 = ['MEAN_TTF']
names_y2 = ['AVAILABILITY', 'MEDIAN_TTF', 'STD_TTF', 'MEAN_TTR',
            'MEDIAN_TTR', 'STD_TTR', 'MEAN_TBF', 'MEDIAN_TBF', 'STD_TBF']
names_X = list(set(df.columns) - set(names_drop) - set(names_y1) - set(names_y2))
names_disc = ['SOLAR_CLIMATE_ZONE', 'VENDOR', 'POWER_NOMINAL_RANGE', 'POWER_MAX_RANGE']
names_median = []
for n in names_X:
    if 'MEDIAN' in n:
        names_median.append(n)
names_acdc = []
for n in names_X:
    if 'AC' in n or 'DC' in n:
        names_acdc.append(n)

In [ ]:
df.drop(names_drop, inplace=True, axis=1)
df.head()

In [ ]:
#### Notizen
Variablen seit 2017:
- DIF
- DNI
- ETR
- FELTT
- GHI
- GNI
- HUMIDITY
- SLP
- TEMP

Wichtige Air Quality Variablen:
- AQI
- pm10
- pm25
- o3
- no2

In [ ]:
### Air Quality

In [ ]:
# read data
aqi = pd.read_csv('aqi_out.csv', sep=',')
aqi.index = aqi.UDID
aqi = aqi[['AQI','pm10','pm25','o3','no2']]

In [ ]:
# join together by index
df = df.join(aqi)

In [ ]:
## Only consider inverters which had at least 1 failure!!!

In [ ]:
df = df[df.AVAILABILITY!=100]


## Exploratory View

In [ ]:
### Boxplot

In [ ]:
# boxplot
print('MEAN')
plt.boxplot(df.MEAN_TTF)
plt.show()
print('MEDIAN')
plt.boxplot(df.MEDIAN_TTF)
plt.show()


# ### Pairplot

# In[14]:


# # Create pairplot for first groups of median variables
# for i in list(range(8,12,4)):
#     pp = sns.pairplot(df[names_y1+names_median[i-3:i]+['POWER_NOMINAL_RANGE']], hue='POWER_NOMINAL_RANGE')
#     pp.savefig('plots/pp_'+str(i)+'.png') 


# ### Correlation Matrix

# In[15]:


corr = df.corr()
sns.set(rc={'figure.figsize':(19,16)})
correlation_matrix = sns.heatmap(corr,
                                 cmap='RdBu',
                                 xticklabels=corr.columns.values,
                                 yticklabels=corr.columns.values,
                                 annot=False)
fig = correlation_matrix.get_figure()
fig.savefig('plots/correlation_matrix_raw.png') 


# In[16]:


# remove other ys
df.drop(names_y2, inplace=True, axis=1)


# In[17]:


# move y variable to first place
cols = list(df)
cols.insert(0, cols.pop(cols.index(names_y1[0])))
# use loc to reorder
df = df.loc[:, cols]


# In[18]:


corr = df.corr()
sns.set(rc={'figure.figsize':(19,16)})
correlation_matrix = sns.heatmap(corr,
                                 cmap='RdBu',
                                 xticklabels=corr.columns.values,
                                 yticklabels=corr.columns.values,
                                 annot=False)
fig = correlation_matrix.get_figure()
fig.savefig('plots/correlation_matrix_raw_sorted.png') 


# In[19]:


# save top 10 correlated X
top_by_lincorr = list(abs(corr[names_y1[0]]).sort_values()[-15:-5].index)


# In[20]:


top_by_lincorr


# In[21]:


# Create pairplot for relevant variables
pp = sns.pairplot(df[names_y1+top_by_lincorr[5:]+['POWER_NOMINAL_RANGE']], hue='POWER_NOMINAL_RANGE')
pp.savefig('plots/pp_top.png') 


# ## Data Cleaning

# In[22]:


# new df object
df_clean = df


# ### Filter known mistakes

# In[23]:


# delete AC mistakes
df_clean = df_clean[df_clean.AC_POWER_MEDIAN.notnull()]


# ### Missing Values

# #### Imputing (by domain knowledge)

# In[24]:


# use raw approximation, if other two values are available for DC
for v in ['MEAN', 'MEDIAN', 'MIN', 'MAX']:
    df_clean['DC_VOLTAGE_'+v].fillna(df_clean['DC_POWER_'+v]/df_clean['DC_CURRENT_'+v], inplace=True)
    df_clean['DC_CURRENT_'+v].fillna(df_clean['DC_POWER_'+v]/df_clean['DC_VOLTAGE_'+v], inplace=True)
    df_clean['DC_POWER_'+v].fillna(df_clean['DC_CURRENT_'+v]*df_clean['DC_VOLTAGE_'+v], inplace=True)


# In[25]:


# user power nominal for few missing values
df_clean['PPEAK'].fillna(df_clean['POWER_NOMINAL'], inplace=True)
df_clean['POWER_MAX'].fillna(df_clean['POWER_NOMINAL'], inplace=True)
df_clean['POWER_MAX_RANGE'].fillna(df_clean['POWER_NOMINAL_RANGE'], inplace=True)


# In[26]:


# transform infs to nan
df_clean = df_clean.replace([np.inf, -np.inf], np.nan)


# #### Imputing (by median)

# In[27]:


check_nulls = df_clean.isnull().sum()
names_mi = list(check_nulls[check_nulls > 0].index)


# In[28]:


for c in names_mi:
    df_clean[c].fillna(df_clean[c].median(), inplace=True)


# In[29]:


# double checking
df_clean.isnull().values.any()


# ### Outlier

# In[30]:


dfo = df_clean.select_dtypes(include=[np.number])


# In[31]:


dfoz = dfo.apply(calc_zscore, axis=0)


# In[32]:


# variables with zero variance
dfoz.fillna(0, inplace=True)


# In[33]:


dfo = dfo[(np.abs(dfoz) < 3).all(axis=1)]


# In[34]:


# join back non-numerical features
df_clean = dfo.join(df_clean.select_dtypes(exclude=[np.number]))


# In[35]:


print(str(len(df) - len(df_clean))+' row(s) got kicked out!')


# ### Dummy vars for non-numeric features

# In[36]:


df_clean = pd.get_dummies(df_clean, drop_first=True)


# In[37]:


df_clean.head()


# ## Feature Cleaning

# In[38]:


X0 = df_clean.drop(names_y1, axis=1)


# ### 1 - Drop features with low variance

# In[39]:


X1 = varianceSelection(X0, THRESHOLD=0.05)  # Variance at least 0.05


# In[40]:


to_drop1 = list(set(X0.columns) - set(X1.columns))


# In[41]:


print(str(len(to_drop1))+' features got kicked out!')


# ### 2 - Drop highly correlated features

# In[42]:


# Create correlation matrix
corr_matrix = X1.corr().abs()


# In[43]:


# Select upper triangle of correlation matrix
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))


# In[44]:


# Find index of feature columns with correlation greater than 0.95
to_drop2 = [column for column in upper.columns if any(upper[column] > 0.95)]


# In[45]:


# Drop features 
X2 = X1.drop(to_drop2, axis=1)


# In[46]:


print(str(len(to_drop2))+' features got kicked out!')


# ### 3 - Drop features which are linearly dependent

# In[47]:


# reduced_form, inds = sympy.Matrix(X2.values).rref()


# In[48]:


# X3 = X2.iloc[:, list(inds)]


# In[49]:


print('We end up with %d features'%X2.shape[1])


# ## Feature Normalization & Scaling

# ### Normalize

# In[50]:


normalizer = Normalizer().fit(X2)
X_norm = pd.DataFrame(normalizer.transform(X2), columns=X2.columns)


# ### Scale

# In[51]:


scaler_mm = MinMaxScaler()
X_scalem = pd.DataFrame(scaler_mm.fit_transform(X2), columns=X2.columns)


# In[52]:


scaler_s = StandardScaler()
X_scales = pd.DataFrame(scaler_s.fit_transform(X2), columns=X2.columns)


# ## Feature Selection

# In[53]:


# define dependent and independent variables
y = df_clean[names_y1[0]]
y_log = np.log(y)
X = X_scalem


# In[54]:


# build 100 trees & show variable importance
clf = RandomForestRegressor(n_estimators=100)
clf = clf.fit(X, y)


# In[55]:


importances = clf.feature_importances_
std = np.std([tree.feature_importances_ for tree in clf.estimators_],
             axis=0)
indices = np.argsort(importances)[::-1]

# Print the feature ranking
print("Feature ranking:")

top_by_rf = []
for f in range(X.shape[1]):
    top_by_rf.append(X.columns[indices[f]])
    print("%d. %s (%f)" % (f + 1, X.columns[indices[f]], importances[indices[f]]))


# **The importances add up to 100% and are calculated based on variance reduction achievable by each feature.**

# In[56]:


# Plot the feature importances of the forest
fig = plt.figure()
plt.title("Feature importances by Random Forest")
plt.bar(range(X.shape[1]), importances[indices],
       color="b", yerr=std[indices], align="center")
plt.xticks(range(X.shape[1]), top_by_rf, rotation=90)
plt.xlim([-1, X.shape[1]])
fig.savefig('plots/rf_feature_importances.png') 
plt.show()


# ## Polynomial Features

# In[57]:


X_reduced_by_rf = X_scalem[top_by_rf[:4]]


# In[58]:


poly = PolynomialFeatures(2)
X_poly = poly.fit_transform(X_reduced_by_rf)


# ## Models

# ### Simple linear regression model: w = beta_hat*X + e

# In[59]:


X_reduced = X[top_by_rf]


# In[60]:


# Create linear regression object with normalized variables as benchmark
linreg = linear_model.LinearRegression(normalize=True)


# In[61]:


# Train the model using the training sets
linreg.fit(X_reduced, y)


# In[62]:


# Make predictions using the testing set
y_lr = linreg.predict(X_reduced)


# In[63]:


# The coefficients
print('Coefficients: \n', linreg.coef_)
# The mean squared error
print("Root Mean squared error (RMSE): %.2f"
      % sqrt(mean_squared_error(y, y_lr)))
# R^2
print('R² score: %.2f' % linreg.score(X_reduced, y))
print('Adj. R² score: %.2f' % (1-(1-linreg.score(X_reduced, y))*(len(y)-1)/(len(y)-X_reduced.shape[1]-1)))


# ### Regression Tree

# In[64]:


# create object
dtr = tree.DecisionTreeRegressor(criterion='mse', max_depth=8)


# In[65]:


# fit data
dtr.fit(X, y)


# In[66]:


# Predict (in-sample)
y_dt = dtr.predict(X)


# In[67]:


dot_data = tree.export_graphviz(dtr, out_file=None,
                                feature_names=list(X.columns),
                                filled=True, rounded=True,
                                special_characters=True) 
graph = graphviz.Source(dot_data) 
graph.render("plots/Regression Tree", format='png') 


# In[68]:


print('RMSE for 1 decision tree: %4f'%sqrt(mean_squared_error(y, y_dt)))


# ### Gradient boosted forest

# In[69]:


# create object
gbr = GradientBoostingRegressor(n_estimators=500, max_depth=8, 
                                min_samples_split=2, learning_rate=0.01,
                                loss='huber')


# In[70]:


# fit data
gbr.fit(X, y)


# In[71]:


# predict (in-sample)
y_gb = gbr.predict(X)


# In[72]:


print('RMSE for Gradient-Boosted Forest (500): %.4f'%sqrt(mean_squared_error(y, y_gb)))


# ### Ada-boosted Trees

# In[73]:


# create object
rng = np.random.RandomState(1)
abr = AdaBoostRegressor(tree.DecisionTreeRegressor(max_depth=8),
                        learning_rate=0.01, loss='linear',
                        n_estimators=500, random_state=rng)


# In[74]:


# fit data
abr.fit(X, y)


# In[75]:


# Predict (in-sample)
y_ab = abr.predict(X)


# In[76]:


print('RMSE for 500 ada boosted-trees: %4f'%sqrt(mean_squared_error(y, y_ab)))


# ### Neural Net

# #### Data

# In[285]:


# data
inputs = Variable(torch.from_numpy(np.array(X, dtype='float32')))
target = Variable(torch.from_numpy(np.array(y, dtype='float32')))


# In[286]:


# Define data loader
batch_size = 32
train_dl = DataLoader(TensorDataset(inputs, target), 
                      batch_size, 
                      shuffle=False,
                      num_workers=6)


# #### Define layers of the net

# In[287]:


nn_model = nn.Sequential(
    nn.Linear(inputs.shape[1], 50),
    nn.Tanh(),
    nn.Linear(50, 50),
    nn.Sigmoid(),
    nn.Linear(50, 100),
    nn.Tanh(),
    nn.Linear(100, 1)
    )


# #### define optimizer & loss function

# In[288]:


opt = torch.optim.SGD(nn_model.parameters(), lr=1e-07)
loss_fn = F.mse_loss


# In[289]:


def RMSELoss(yhat,y):
    return torch.sqrt(torch.mean((yhat-y)**2))

loss_fn = RMSELoss


# #### train the model

# In[290]:


# Define a utility function to train the model
def fit(df, num_epochs, model, loss_fn, opt):
    for epoch in range(num_epochs):
        for xb,yb in df:
            # Generate predictions
            pred = model(xb)
            loss = loss_fn(pred, yb)
            # Perform gradient descent
            opt.zero_grad()
            loss.backward()
            opt.step()
        if epoch%10==0:
            print("Epoch %s: Loss %s"%(epoch, loss.data.item()))
    print('Training loss: ', loss_fn(model(inputs), target).data.item())


# In[291]:


# Train the model for X epochs
fit(train_dl, 50, nn_model, loss_fn, opt)


# In[292]:


preds = nn_model(inputs)


# In[293]:


# save predictions as array
y_nn = preds.detach().numpy().squeeze()


# In[294]:


print('RMSE for Neural Net: %4f'%sqrt(mean_squared_error(y, y_nn)))


# In[295]:


plt.hist(y_nn)


# ### Ensemble Predictions

# In[ ]:


y_ens = np.mean([y_dt, y_gb, y_ab, y_nn], axis=0)


# In[ ]:


print('RMSE for Ensemble Predictions: %4f'%sqrt(mean_squared_error(y, y_ens)))


# ## Evaluation

# ### Descriptive plots

# In[ ]:


# create data_frame of predictions and true values
df_preds = pd.DataFrame(data={'y_true':y, 'y_lr':y_lr, 'y_dt':y_dt, 
                              'y_gb':y_gb, 'y_ab':y_ab, 'y_nn':y_nn})


# In[ ]:


# add power range
df_preds = df_preds.join(df.POWER_NOMINAL_RANGE)


# In[ ]:


# pairplots
ppp = sns.pairplot(df_preds, hue='POWER_NOMINAL_RANGE')
ppp.savefig('plots/pp_predictions.png') 


# In[ ]:


# coordinates for origin line
ol = list(range(0, 40000, 5000))


# In[ ]:


olp = sns.lmplot(x='y_nn', y='y_true', data=df_preds, fit_reg=False, 
           hue='POWER_NOMINAL_RANGE', legend=False, palette="Set1")
plt.plot(ol, ol, linewidth=2)
plt.legend(loc='lower right')
olp.savefig('plots/true_vs_predictions.png') 


# In[ ]:


resp = sns.residplot(x='y_gb', y='y_true', data=df_preds)
resp.figure.savefig('plots/residuals.png') 


# In[ ]: